In [5]:
import sys
sys.path.append('..')
import os
from keras.models import Model
from l3embedding.model import load_model
from l3embedding import vision_model, audio_model
import gzip, bz2

In [ ]:
output_dir = "/scratch/jtc440/openl3_models"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)


In [6]:
def compress_gzip(filepath):
    with open(filepath, 'rb') as f:
        data = f.read()
        
    with gzip.open(filepath + '.gz', 'w') as f:
        f.write(data)
        
def compress_gzip(filepath):
    with open(filepath, 'rb') as f:
        data = f.read()
        
    with gzip.open(filepath + '.gz', 'w') as f:
        f.write(data)
        
def compress_bz2(filepath):
    with open(filepath, 'rb') as f:
        data = f.read()
            
    with bz2.BZ2File(filepath + '.bz2', 'w') as f:
        f.write(data)

In [28]:
weights_path = '/scratch/jtc440/l3_output/converted_models/embedding/music/cnn_L3_kapredbinputbn/20180905220149/model_best_valid_accuracy.h5'
weights_path = '/scratch/jtc440/l3_output/converted_models/embedding/music/cnn_L3_melspec1/20180905154214/model_best_valid_accuracy.h5'
weights_path = '/scratch/jtc440/l3_output/converted_models/embedding/music/cnn_L3_melspec2/20180906172059/model_best_valid_accuracy.h5'
weights_path = '/scratch/jtc440/l3_output/converted_models/embedding/environmental/cnn_L3_kapredbinputbn/20180905221614/model_best_valid_accuracy.h5'
weights_path = '/scratch/jtc440/l3_output/converted_models/embedding/environmental/cnn_L3_melspec1/20180905195208/model_best_valid_accuracy.h5'
weights_path = '/scratch/jtc440/l3_output/converted_models/embedding/environmental/cnn_L3_melspec2/20180905191651/model_best_valid_accuracy.h5'


model_type = weights_path.split('/')[7]

if 'kapre' in weights_path:
    input_repr = "linear"
elif 'melspec1' in weights_path:
    input_repr = "mel128"
elif 'melspec2' in weights_path:
    input_repr = "mel256"

subset = weights_path.split('/')[6]
if subset[0] == 'e':
    subset = 'env'

m, inputs, output = load_model(weights_path, model_type, return_io=True)                                     
x_i, x_a = inputs
image_model_output = m.get_layer('vision_model').get_layer('vision_embedding_layer').output
audio_model_output = m.get_layer('audio_model').get_layer('audio_embedding_layer').output

image_embed_model = Model(inputs=x_i, outputs=image_model_output)
audio_embed_model = Model(inputs=x_a, outputs=audio_model_output)

audio_output_path = os.path.join(output_dir, 'openl3_audio_{}_{}.h5'.format(input_repr, subset))
image_output_path = os.path.join(output_dir, 'openl3_image_{}_{}.h5'.format(input_repr, subset))

image_embed_model.save_weights(image_output_path)
audio_embed_model.save_weights(audio_output_path)

compress_gzip(image_output_path)
compress_bz2(image_output_path)
compress_gzip(audio_output_path)
compress_bz2(audio_output_path)

/beegfs/jtc440/miniconda3/envs/l3embedding-new/lib/python3.6/site-packages/librosa/filters.py:261: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '
